In [0]:
from bs4 import BeautifulSoup
import pymysql.cursors
import time
import pandas as pd
import requests
import json

In [0]:
# 대학 코드 수집
url = 'http://ysweb.yonsei.ac.kr:8888/curri120601/curri_new.jsp#top'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
college_tags = soup.select('#OCODE1 option')
college_names = []
college_nos = []
for college_tag in college_tags:
    college_names.append(college_tag.text)
    college_nos.append(college_tag.attrs['value'])

In [0]:
pd.DataFrame([college_names, college_nos]).transpose()

In [0]:
## 마일리지 페이지를 띄우기 위한 정보를 가져오는 크롤러
def searchkey_crawler(school_no,year,semester):
    url = "http://ysweb.yonsei.ac.kr:8888/DataAgent?pgm=/curri120601/curri_Data&DmlGb=get_list&ocode0=s1&ocode1="+school_no+"&s2=all&schHakjum=all&hy="+year+"&hg="+semester+"&lang=0&filterscount=0&groupscount=0&pagenum=0&pagesize=15&recordstartindex=0&recordendindex=34.56"
    header =  {"Referer" : "http://ysweb.yonsei.ac.kr:8888/curri120601/curri_new.jsp"}
    response = requests.get(url, headers = header)
    HAKBBSBB_lst = []
    for i in response.json()['Data']:
        searchdict = {'yshs_domain': i['DOMAIN'],
                      'yshs_hyhg': i['HYHG'],
                      'yshs_hakno': i['HAKBBSBB'].split('-')[0],
                      'yshs_bb': i['HAKBBSBB'].split('-')[1],
                      'yshs_sbb': i['HAKBBSBB'].split('-')[2]}
        HAKBBSBB_lst.append(searchdict)
    return(HAKBBSBB_lst)

In [0]:
## 마일리지 페이지에서 정보를 가져오는 크롤러
def page_crawler(params):
    url= "http://ysweb.yonsei.ac.kr:8888/curri120601/curri_pop_mileage_result01.jsp"
    response = requests.post(url=url,data=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    meta = []
    if len(soup.findAll('tr')) < 4:  # 마일리지 정보가 없는 수업 구분
        return [], []
    for i in soup.findAll('tr')[4].findAll('td'):
        meta.append(i.text)
    df_lst = []
    for i in soup.findAll('tr')[6].findAll('tr'):
        row = [params['yshs_domain'],params['yshs_hyhg'],params['yshs_hakno'],params['yshs_bb'],params['yshs_sbb']]
        for j in i.findAll('td'):
            row.append(j.text)
        df_lst.append(row)
    return meta, df_lst

In [0]:
years = ['2016', '2017', '2018', '2019']
semesters = ['1', '2']

In [0]:
conn = pymysql.connect(
    host='13.125.25.36', 
    port=3306, 
    user='root',
    passwd='mypassword',
    db='testdb',
    charset='utf8',
)

try:
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE meta_table (
                hyhg INT NOT NULL,
                course_code VARCHAR(255) NOT NULL,
                course_title VARCHAR(255) NOT NULL,
                credit INT NOT NULL,
                instructor VARCHAR(255) NOT NULL,
                time VARCHAR(255) NOT NULL,
                room VARCHAR(255) NOT NULL,
                quota INT NOT NULL,
                participants VARCHAR(255) NOT NULL,
                major_quota INT NOT NULL, 
                second_major VARCHAR(255) NOT NULL,
                grade_1 INT NOT NULL,
                grade_2 INT NOT NULL, 
                grade_3 INT NOT NULL, 
                grade_4 INT NOT NULL, 
                exchange_student VARCHAR(255) NOT NULL, 
                max_mileage INT NOT NULL, 
                min_result INT NOT NULL, 
                max_result INT NOT NULL, 
                average FLOAT NOT NULL
               )'''
        cursor.execute(sql)
    conn.commit()
except:
    print('db exist')
finally:
    conn.close()

In [0]:
conn = pymysql.connect(
    host='13.125.25.36', 
    port=3306, 
    user='root',
    passwd='mypassword',
    db='testdb',
    charset='utf8'
)

try:
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE mile_table (
                domain VARCHAR(255) NOT NULL, 
                hyhg INT NOT NULL, 
                hakno VARCHAR(255) NOT NULL, 
                bb INT NOT NULL, 
                sbb INT NOT NULL, 
                rank INT NOT NULL, 
                mileage INT NOT NULL, 
                major VARCHAR(255) NOT NULL,
                double_major VARCHAR(255) NOT NULL, 
                enrolled_courses INT NOT NULL, 
                graduation VARCHAR(255) NOT NULL, 
                first_enroll VARCHAR(255) NOT NULL, 
                credits_rate FLOAT NOT NULL, 
                previous_credits_rate FLOAT NOT NULL, 
                grade INT NOT NULL, 
                enrolled VARCHAR(255) NOT NULL, 
                etc VARCHAR(255) NULL
               )'''
        cursor.execute(sql)
    conn.commit()
except:
    print('db exist')
finally:
    conn.close()


In [0]:
def meta_preprocess(lst):
    major_split = lst[9].split(' ')
    lst[9] = major_split[0]
    lst.insert(10, major_split[1][1:-1])
    return tuple(lst)

In [0]:
def mile_preprocess(lst):
    major_split = lst[7].split(' ')
    lst[7] = major_split[0]
    lst.insert(8, major_split[1][1:-1])
    return tuple(lst)

In [0]:
def insert_meta(tpl):
    conn = pymysql.connect(
        host='13.125.25.36', 
        port=3306, 
        user='root',
        passwd='mypassword',
        db='testdb', 
        charset='utf8'
    )
    try:
        with conn.cursor() as cursor:
            sql = 'INSERT INTO meta_table VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
            cursor.execute(sql, tpl)
        conn.commit()
    except:
        print('INSERT INTO meta_table VALUES failed')
    finally:
        conn.commit()

In [0]:
def insert_mile(tpl):
    conn = pymysql.connect(
        host='13.125.25.36', 
        port=3306, 
        user='root',
        passwd='mypassword',
        db='testdb', 
        charset='utf8'
    )
    try:
        with conn.cursor() as cursor:
            sql = 'INSERT INTO mile_table VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
            cursor.execute(sql, tpl)
        conn.commit()
    except:
        print('INSERT INTO mile_table VALUES failed')
    finally:
        conn.commit()

In [0]:
for year in years:
    for semester in semesters:
        for college_no in college_nos:
            print(year, semester, college_no) # 몇년도 몇학기 어느 대학 크롤링 중인지 확인하기 위함
            searchparams = searchkey_crawler(college_no, year, semester)
            for i in searchparams:
                crawl = page_crawler(i)
                if crawl == -1:
                    continue
                meta_lst,mile_lst = crawl
                if len(meta_lst) < 18:
                    continue
                insert_meta(meta_preprocess([year+semester]+meta_lst))
                for e in mile_lst:
                    insert_mile(mile_preprocess(e))